Skin cancer detection

In [ ]:
from keras_preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
import os, shutil
import pandas as pd

import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
sns.set_theme(style="whitegrid")

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import load_digits, load_sample_images

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
import seaborn as sns

import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras import datasets, layers, models

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#%load_ext autoreload
#%autoreload 2

In [ ]:
# Directory path
train_data_dir = 'dataskin/Train'
test_data_dir = 'dataskin/Test'

# Get all the data in the directory data/validation (132 images), and reshape them
test_generator = ImageDataGenerator().flow_from_directory(
        test_data_dir, batch_size=118)

# Get all the data in the directory data/train (790 images), and reshape them
train_generator = ImageDataGenerator().flow_from_directory(
        train_data_dir, batch_size=2239)

# Create the datasets
train_images_fs, train_labels = next(train_generator)
test_images_fs, test_labels = next(test_generator)
#val_images, val_labels = next(val_generator)

In [ ]:
train_images

In [ ]:
# Preview an image
array_to_img(train_images[0])

In [ ]:
# Preview an image
array_to_img(test_images[0])

In [ ]:
train_labels.shape

In [ ]:
test_labels.shape

In [ ]:
test_images.shape

In [ ]:
train_images.shape

In [ ]:
train_labels

In [ ]:
type(train_labels)

In [ ]:
type(train_images)

In [ ]:
dataframe=pd.DataFrame(train_labels, columns=['actinic_keratosis', 'basal_cell_carcinoma',\
            'dermatofibroma', 'melanoma', 'nevus', 'pigmented_benign_keratosis',\
            'seborrheic_keratosis', 'squamous_cell_carcinoma', 'vascular_lesion'])

In [ ]:
dataframe

In [ ]:
#dataframe.hist()

In [ ]:
sums=dataframe.sum(axis=0)

In [ ]:
sums=pd.DataFrame(sums)

In [ ]:
sums

In [ ]:
sums['%'] = ((sums[0] / 2239*100).round(2).astype(str) + '%')

In [ ]:
sums

In [ ]:
#Change this into seaborn to make it even cooler
sums.plot(kind='bar');

In [ ]:
train_generator.class_indices 

Defining a function to visualize the results.

In [ ]:
"""def visualize_training_results(results):
    history = results.history
    plt.figure()
    plt.plot(history['val_loss'])
    plt.plot(history['loss'])
    plt.legend(['val_loss', 'loss'])
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()
    
    plt.figure()
    plt.plot(history['val_accuracy'])
    plt.plot(history['accuracy'])
    plt.legend(['val_accuracy', 'accuracy'])
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.show()"""

Look at lab on https://github.com/learn-co-curriculum/dsc-image-classification-with-mlps-lab/tree/solution
might need to do some one hot encoding.
Also, just like it shows there, as a naive model maybe decrease the number of layers and keep epochs at like 5.
If results are the same, try to understand why the loss still showes such high peaks.And why the accuracy drops like that.
Might also be worth doing CNN from the start with 2D layers instead of this basic NN.

Let us forget about all of this and try with the 2D actual convolutional neural network for images. The perfmormance there might be different.

We can also do something else which is group together the types of skin abnormalities that are cancerous and the ones that are not, reducing the classes to 2 and making this a binary classification problem.
I will do that later on. For now let's see what happens with a convolutional neural network, which is what I should be doing anyway.

Here also definitely preview one image per class and explain a bit about each class.

## Preprocessing

In [ ]:
# Directory path
train_data_dir = 'dataskin/Train'
test_data_dir = 'dataskin/Test'
#val_data_dir = 'dataset/validation_set'

# Get all the data in the directory data/validation (118 images), and reshape them
test_gen= ImageDataGenerator().flow_from_directory(
        test_data_dir, target_size=(32, 32), batch_size=118)

# Get all the data in the directory data/train (2239 images), and reshape them
train_gen = ImageDataGenerator().flow_from_directory(
        train_data_dir, target_size=(32, 32), batch_size=2239)

# Create the datasets
train_img, train_lab = next(train_gen)
test_img, test_lab = next(test_gen)

In [ ]:
# Explore your dataset again
m_train = train_img.shape[0]
num_px = train_img.shape[1]
m_test = test_img.shape[0]
#m_val = val_images.shape[0]

print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))
#print ("Number of validation samples: " + str(m_val))
print ("train_images shape: " + str(train_img.shape))
print ("train_labels shape: " + str(train_lab.shape))
print ("test_images shape: " + str(test_img.shape))
print ("test_labels shape: " + str(test_lab.shape))
#print ("val_images shape: " + str(val_images.shape))
#print ("val_labels shape: " + str(val_labels.shape))

In [ ]:
train_lab.shape[0]

In [ ]:
#train_img = train_img.reshape(train_img.shape[0], -1)
#test_img = test_img.reshape(test_img.shape[0], -1)
#val_img = val_images.reshape(val_images.shape[0], -1)

#print(val_img.shape)

As we can see the RGB values of each pixel are not scaled, we are going to scale their value between 0 and 1 by dividing each pixel value by 225,
and we are also going to reshape the data in a format that can be fed into the model.
We are going to put these two preprocessing steps together with a pipeline.

#### Here I am going to create my first pipeline 
to use to scale and reshape the data once I have loaded it
Or possibly even a function to load it.

to standardize the pixel values:

In [ ]:
def scale_pixels(train_img, test_img, train_lab, test_lab):
    train_img /= 255.
    test_img /= 255.
    return train_img, test_img

In [ ]:
# Instantiate transformer
pix_scaler = FunctionTransformer(scale_pixels)

In [ ]:
pix_scaler.fit_transform(train_img, test_img, train_lab, test_lab)

In [ ]:
#pix_scaler.transform(train_img, test_img)

In [ ]:
#from sklearn.preprocessing import StandardScaler
#StandardScaler().fit_transform(train_img)

In [ ]:
test_img.shape

In [ ]:
train_img.shape

In [ ]:
# Instantiate StandardScaler
#scaler = MinMaxScaler()
# Transform the training and test sets
#scaled_data_train = scaler.fit_transform(train_img)
#scaled_data_test = scaler.transform(test_img)

# Convert into a DataFrame
#scaled_df_train = pd.DataFrame(scaled_data_train, columns=X_train.columns)
#scaled_df_train.head()

In [ ]:
#scale_pixels(train_img, test_img)

In [ ]:
#train_img

To change the shape of the labels:

In [ ]:
def reshape (train_lab, test_lab): #(train_img, test_img, train_lab, test_lab):
    m=train_lab.shape[0]
    n=test_lab.shape[0]
    train_lab = np.reshape(train_lab[:,0], (m,1))
    test_lab = np.reshape(test_lab[:,0], (n,1))
    return train_lab, test_lab

In [ ]:
# Instantiate transformer
shaper = FunctionTransformer(reshape)

In [ ]:
print(train_lab.shape)
print(test_lab.shape)

In [ ]:
#train_y, test_y=reshape(train_lab, test_lab)

In [ ]:
shaper.fit_transform(train_lab, test_lab)#(train_img, test_img, train_lab, test_lab)

In [ ]:
print(train_lab.shape)
print(test_lab.shape)

In [ ]:
train_y = np.reshape(train_lab[:,0], (2239,1))
test_y = np.reshape(test_lab[:,0], (118,1))

In [ ]:
print(train_y.shape)
print(test_y.shape)

In [ ]:
ct=make_column_transformer(
    (pix_scaler, ['img_scaled']),
    (shaper, ['labels_reshaped']))

In [ ]:
ct.fit_transform(train_img, test_img, train_lab, test_lab)

In [ ]:
# Build a pipeline with StandardScaler and KNeighborsClassifier
scale_pipeline = Pipeline([('pixel scaler', pix_scaler(train_img, test_img)),
        ('changing shape', shaper(train_lab, test_lab))])

In [ ]:
scale_pipeline.fit_transform(train_img, test_img, train_lab, test_lab)

In [ ]:
def visualize_training_results(results):
    history = results.history
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))
    ax1.plot(history['val_acc'])
    ax1.plot(history['acc'])
    ax1.legend(['val_acc', 'acc'], loc='best')
    ax1.set_title('Accuracy')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Accuracy')
    
    ax2.plot(history['val_loss'])
    ax2.plot(history['loss'])
    ax2.legend(['val_loss', 'loss'], loc='best')
    ax2.set_title('Loss')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Loss')

Creating the empty dataframe where we will store the results:

In [ ]:
# create an Empty DataFrame object
df = pd.DataFrame()

In [ ]:
def model_results(results, model_name, df):
    visualize_training_results(results)
    df1 = pd.DataFrame({'Name': [model_name],
                        'accuracy train': [round(results.history['acc'][-1],3)] ,
                        'accuracy validation': [round(results.history['val_acc'][-1],3 )],
                        'loss train': [round(results.history['loss'][-1],3)],
                        'loss validation': [round(results.history['val_loss'][-1],3)]})
    df=df.append(df1)
    fig = plt.subplots(figsize=(12,6))
    
    plt.subplot(1,2,1)
    x=df['Name']
    y=df['accuracy train']
    plt.plot(x, y, color='r')
    x1=df['Name']
    y1=df['accuracy validation']
    plt.plot(x1, y1, color='g')
    plt.xticks(x, labels=df['Name'], rotation='vertical')
    plt.title('Learning Curve Accuracy')
    plt.legend()
#    fig = plt.subplots(figsize=(6,6)) 
    plt.subplot(1,2,2)
    x=df['Name']
    y=df['loss train']
    plt.plot(x, y, color='r')
    x1=df['Name']
    y1=df['loss validation']
    plt.plot(x1, y1, color='g')
    plt.xticks(x, labels=df['Name'], rotation='vertical')
    plt.title('Learning Curve Loss')
    plt.legend()
#    sns.pointplot(x=df['Name'], y=df['accuracy train'], 
#                linestyles='dotted',data=df, ax=ax1)
#                  hue=df[['accuracy train', 'accuracy validation']])
#    ax1.set_xticklabels(labels=df['Name'], rotation=90)

#    sns.pointplot(x=df['Name'], y=df['accuracy validation'],color='r',
#                linestyles='dotted',data=df, ax=ax1, label='validation')
#                  ,  hue=df['accuracy train', 'accuracy validation'].apply(tuple, axis=1))
#    ax1.set_xticklabels(labels=df['Name'], rotation=90)
#    ax1.set(xlabel='Model')
#    ax1.set_title('Learning Curve Accuracy')

#    plt.legend(handles=[df,df], loc='best')
#    ax1.legend(['accuracy train', 'accuracy validation'])
#    ax1.legend(handles=ax1.lines[::len(df1)+1], labels=['train', 'validation'])
#    sns.pointplot(x=df['Name'], y=df['loss train'],color='b',
#                linestyles='dotted',data=df, ax=ax2)
#    ax2.set_xticklabels(labels=df['Name'], rotation=90)
#    sns.pointplot(x=df['Name'], y=df['loss validation'],color='r',
#                  linestyles='dotted',data=df, ax=ax2)
#    ax2.set_xticklabels(labels=df['Name'], rotation=90)
#    ax2.legend(labels=['loss train', 'loss validation'], loc='best')
#    ax2.legend()
#    ax2.set_title('Learning Curve Loss')
#    ax2.set(xlabel='Model')
    plt.show()
    return df

In [ ]:
df=model_results(history1, '32x32 1st model', df)
df

In [ ]:
#visualize_training_results(history)

### Trying to pick a Naive model

Trying with a simple model:

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(10, (3, 3), activation='relu',
                        input_shape=(32 ,32,  3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(9, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

And doing only 10 epochs:

In [ ]:
history1 = model.fit(train_img,
                    train_y,
                    epochs=10,
                    batch_size=10, validation_split=0.3)

In [ ]:
df=model_results(history1, '32x32 1st model', df)
df

In [ ]:
# Build a pipeline with StandardScaler and KNeighborsClassifier
#scaled_pipeline_1 = Pipeline([('ssc', StandardScaler()), ('KNN', KNeighborsClassifier())])

In [ ]:
"""# Fit the training data to pipeline
scaled_pipeline_1.fit(X_train, y_train)

# Print the accuracy on test set
scaled_pipeline_1.score(X_test, y_test)"""

Let me try with a larger batch size:

In [ ]:
history2 = model.fit(train_img,
                    train_y,
                    epochs=10,
                    batch_size=50, validation_split=0.2)

In [ ]:
df=model_results(history2, '32x32 2nd model, larger batch', df)
df

Another attempt: all the previous parameters but making the last dense layer smaller.

In [ ]:
model2 = models.Sequential()
model2.add(layers.Conv2D(10, (3, 3), activation='relu',
                        input_shape=(32 ,32,  3)))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Flatten())
model2.add(layers.Dense(10, activation='relu'))
model2.add(layers.Dense(9, activation='softmax'))

model2.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
history3 = model2.fit(train_img,
                    train_y,
                    epochs=10,
                    batch_size=50, validation_split=0.2)

In [ ]:
df=model_results(history3, '32x32 3rd model, < nrns last layer', df)
df

Even less Neurons:

In [ ]:
model3 = models.Sequential()
model3.add(layers.Conv2D(5, (3, 3), activation='relu',
                        input_shape=(32 ,32,  3)))
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Flatten())
model3.add(layers.Dense(5, activation='relu'))
model3.add(layers.Dense(9, activation='softmax'))

model3.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
history4 = model3.fit(train_img,
                    train_y,
                    epochs=10,
                    batch_size=50, validation_split=0.2)

In [ ]:
df=model_results(history4, '32x32 4th model, < neurons', df)
df

#### No sorry one more thing. Even less resolution images.

In [ ]:
# Directory path
train_data_dir = 'dataskin/Train'
test_data_dir = 'dataskin/Test'
#val_data_dir = 'dataset/validation_set'

# Get all the data in the directory data/validation (118 images), and reshape them
test_gen= ImageDataGenerator().flow_from_directory(
        test_data_dir, target_size=(8, 8), batch_size=118)

# Get all the data in the directory data/train (2239 images), and reshape them
train_gen = ImageDataGenerator().flow_from_directory(
        train_data_dir, target_size=(8, 8), batch_size=2239)

# Create the datasets
train_img, train_lab = next(train_gen)
test_img, test_lab = next(test_gen)

In [ ]:
train_img.shape

In [ ]:
test_img.shape

In [ ]:
train_y = np.reshape(train_lab[:,0], (2239,1))
test_y = np.reshape(test_lab[:,0], (118,1))

In [ ]:
model5 = models.Sequential()
model5.add(layers.Conv2D(3, (3, 3), activation='relu',
                        input_shape=(8 ,8,  3)))
model5.add(layers.MaxPooling2D((2, 2)))

model5.add(layers.Flatten())
model5.add(layers.Dense(9, activation='softmax'))

model5.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
history5 = model5.fit(train_img,
                    train_y,
                    epochs=10,
                    batch_size=10, validation_split=0.3)

In [ ]:
df=model_results(history5, '8x8 5th model', df)
df

Let me try to decrease the batch size already from the import, see if is changes anything.

In [ ]:
# Directory path
train_data_dir = 'dataskin/Train'
test_data_dir = 'dataskin/Test'
#val_data_dir = 'dataset/validation_set'

# Get all the data in the directory data/validation (118 images), and reshape them
test_gen= ImageDataGenerator().flow_from_directory(
        test_data_dir, target_size=(8, 8), batch_size=5)

# Get all the data in the directory data/train (2239 images), and reshape them
train_gen = ImageDataGenerator().flow_from_directory(
        train_data_dir, target_size=(8, 8), batch_size=20)

# Create the datasets
train_img, train_lab = next(train_gen)
test_img, test_lab = next(test_gen)

In [ ]:
train_y = np.reshape(train_lab[:,0], (20,1))
test_y = np.reshape(test_lab[:,0], (5,1))

In [ ]:
history6 = model5.fit(train_img,
                    train_y,
                    epochs=10,
                    batch_size=10, validation_split=0.3)

In [ ]:
df=model_results(history6, '8x8 5th model < batch on import', df)
df

The good news is that even if the results change a lot in the first part where I do regualr NN, every time I run the code, they don't change so radically with the CNN.
I will keep track a bit but overall it seems like I can stop worrying about reproducibe results, if I use CNN right away.

## Starting to improve our model:

Taking from the german signs notebook

Nope. It's in AWS Sagemaker. But I can look back at it and use either a simple pipeline for loading and changing the size of the images, or for images augmentation to balance the classes of the different skin anomalies.
<br>A simple pipeline can even be "load, normalize, augment".

#### To improve our model we will take several steps:
   - normalize the data 
   - increase image resolution
   - add more epochs
   - add more layers
   - increase the batch size
   - change activation function and optimization
    
After we have done all this we will select the best performing model and we will also group up into 2 classes, cancerous and benign growth too.
<br>Maybe I should do that now actually.
<br>No I will do it after this first selection so I can say I have a good model to predict which of the 9 classes with this much accuracy.
<br>And then introduce the concept of two classes and decreasing recall.
<br>After that anyhow, do another round of tuning for the model, this time using:
 - drop out layers
 - regularization
 - normalization
 - data augmentation
 - balancing classes with data augmentation
 

### Improving the model with 9 classes:

Restart with an empty dataframe because while before we were looking for the lowest performance model now we are looking to improve it.

In [ ]:
df = pd.DataFrame()

Let us start by visualizing again the results from what we picked to be our naive model:

In [ ]:
df=model_results(history5, 'Naive', df)
df

#### Normalize the data:

To standardize the pixel values:

In [ ]:
train_img /= 255.
test_img /= 255.

In [ ]:
train_img

In [ ]:
history52 = model5.fit(train_img,
                    train_y,
                    epochs=10,
                    batch_size=10, validation_split=0.3)

In [ ]:
df=model_results(history52, '8x8 5th model scaled pixels', df)
df

#### Increasing Image resolution:

In [ ]:
# Directory path
train_data_dir = 'dataskin/Train'
test_data_dir = 'dataskin/Test'
#val_data_dir = 'dataset/validation_set'

# Get all the data in the directory data/validation (118 images), and reshape them
test_gen= ImageDataGenerator().flow_from_directory(
        test_data_dir, target_size=(64, 64), batch_size=118)

# Get all the data in the directory data/train (2239 images), and reshape them
train_gen = ImageDataGenerator().flow_from_directory(
        train_data_dir, target_size=(64, 64), batch_size=2239)

# Create the datasets
train_img, train_lab = next(train_gen)
test_img, test_lab = next(test_gen)
#normalizing again
train_img /= 255.
test_img /= 255.

In [ ]:
train_y = np.reshape(train_lab[:,0], (2239,1))
test_y = np.reshape(test_lab[:,0], (118,1))

In [ ]:
model6 = models.Sequential()
model6.add(layers.Conv2D(3, (3, 3), activation='relu',
                        input_shape=(64, 64, 3)))
model6.add(layers.MaxPooling2D((2, 2)))

model6.add(layers.Flatten())
model6.add(layers.Dense(9, activation='softmax'))

model6.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
history6b = model6.fit(train_img,
                    train_y,
                    epochs=10,
                    batch_size=10, validation_split=0.3)

In [ ]:
df=model_results(history6b, '64x64 6th model', df)
df

Let us try increasing the size of the images imported further. Let us go all the way to the actually full size which is 256. These are already loaded at the beginning and are sotred in train_images_fs, test_images_fs.

In [ ]:
#normalizing again
train_images_fs /= 255.
test_images_fs /= 255.

In [ ]:
model6b = models.Sequential()
model6b.add(layers.Conv2D(3, (3, 3), activation='relu',
                        input_shape=(256, 256, 3)))
model6b.add(layers.MaxPooling2D((2, 2)))

model6b.add(layers.Flatten())
model6b.add(layers.Dense(9, activation='softmax'))

model6b.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
history6c = model6b.fit(train_images_fs,
                    train_y,
                    epochs=10,
                    batch_size=10, validation_split=0.3)

In [ ]:
df=model_results(history6c, '256x256 6th model', df)
df

Let us see what happens with the full size.

#### Increasing number of epochs:

In [ ]:
history7 = model6.fit(train_img,
                    train_y,
                    epochs=30,
                    batch_size=10, validation_split=0.3)

In [ ]:
df=model_results(history7, '64x64 6th model, 30 epochs', df)
df

#### Adding more layers:

In [ ]:
model7 = models.Sequential()
model7.add(layers.Conv2D(10, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))
model7.add(layers.MaxPooling2D((2, 2)))

model7.add(layers.Conv2D(10, (4, 4), activation='relu'))
model7.add(layers.MaxPooling2D((2, 2)))

model7.add(layers.Conv2D(32, (3, 3), activation='relu'))
model7.add(layers.MaxPooling2D((2, 2)))

model7.add(layers.Flatten())
model7.add(layers.Dense(32, activation='relu'))
model7.add(layers.Dense(9, activation='softmax'))

model7.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
history8 = model7.fit(train_img,
                    train_y,
                    epochs=30,
                    batch_size=10, validation_split=0.3)

In [ ]:
df=model_results(history8, '64x64 7th model, 30 epochs, more layers', df)
df

#### Adding more layers and higher resolution images

In [ ]:
model7b = models.Sequential()
model7b.add(layers.Conv2D(10, (3, 3), activation='relu',
                        input_shape=(256 ,256,  3)))
model7b.add(layers.MaxPooling2D((2, 2)))

model7b.add(layers.Conv2D(10, (4, 4), activation='relu'))
model7b.add(layers.MaxPooling2D((2, 2)))

model7b.add(layers.Conv2D(32, (3, 3), activation='relu'))
model7b.add(layers.MaxPooling2D((2, 2)))

model7b.add(layers.Flatten())
model7b.add(layers.Dense(32, activation='relu'))
model7b.add(layers.Dense(9, activation='softmax'))

model7b.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
history8b = model7b.fit(train_images_fs,
                    train_y,
                    epochs=30,
                    batch_size=10, validation_split=0.3)

In [ ]:
df=model_results(history8b, '256x256 7th model', df)
df

#### Increase the batch size: 50

In [ ]:
history9 = model7.fit(train_img,
                    train_y,
                    epochs=30,
                    batch_size=50, validation_split=0.3)

In [ ]:
df=model_results(history9, '64x64 7th model, batch size=50', df)
df

The truth is that just getting one result for each of these attempts doesn't really tell us too much about the improvement or not of our model, because of the randomness of the results.
What would be more appropriate is for each change we do (in the size of the images, the number of epochs, the batch size) to try a few different values to be able to really get a sense of in which direction our model is moving with that change.
This would be very time consuming to do by hand, so the next step we are goign to take is building a grid search for these parameteres to get a better answer in terms of which ones are the optimal charateristic of our model, which gives us the best accuracy in determining the classification of the skin anomaly.

### Grid Search batch size and epochs:

Grid search is a model hyperparameter optimization technique.
In scikit learn this is a class, and when running a search with this class, we must input a dictionary of hyperparameters to evaluate in the param_grid argument. 
This dictionary contains the model parameter name and an array of values to try.

The GridSearchCV essentially builds and then evaluates one model for each combination of parameters. 
To evaluate each individual model, a 3-fold cross validation is used.

Even if we might have seen a better result with higher resolution images, we will keep the 64x64 ones for the grid search to limit the running time of our code.
Once we  find out from the GridSearch what are the best parameteres we can use those (running just one single model) with higher resolution images.

In [ ]:
# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
 # create model
#    model = Sequential()
#    model.add(Dense(12, input_shape=(8,), activation='relu'))
#    model.add(Dense(1, activation='sigmoid'))
    model7 = models.Sequential()
    model7.add(layers.Conv2D(10, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))
    model7.add(layers.MaxPooling2D((2, 2)))
    
    model7.add(layers.Conv2D(10, (4, 4), activation='relu'))
    model7.add(layers.MaxPooling2D((2, 2)))

    model7.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model7.add(layers.MaxPooling2D((2, 2)))

    model7.add(layers.Flatten())
    model7.add(layers.Dense(32, activation='relu'))
    model7.add(layers.Dense(9, activation='softmax'))


# Compile model
    model7.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
    return model
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)
# load dataset
#dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
#X = dataset[:,0:8]
#Y = dataset[:,8]
X=train_img
Y=train_y
# create model
model = KerasClassifier(model=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid)#, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
best_params=paramsgrid_result.best_params_

In [ ]:
# Figure out here exactly how to extract each, the epochs and batch size.

Now we can run again a model with the higher resolution images, using the best parameters we obtained from the grid search.

In [ ]:
history10 = model7b.fit(train_images_fs,
                    train_y,
                    epochs=best_epochs,
                    batch_size=best_batch, validation_split=0.3)

In [ ]:
df=model_results(history10, 'best model 1st Gridsearch', df)
df

### Grid Search Activation Functions and Optimizations:

We can run again a GridSearch, but this time instead of trying different numbers of epochs and batch sizes, we will try different activation functions and optimizations. Here are lists of both, and as we can see from the number of options, GridSearch is a good idea to shorten the time in trying all the different possible combinations.

#### Changing activation functions and optimizations:
Some of the options for activation functions are:
<br>relu
<br>swish
<br>softmax
<br>leaky relu
<br>sigmoid

<br>for optimizers:
<br>adam, sgd, 
<br>with optimizer I can also tweak the learning rate

<br>for loss functions:
categorical_crossentropy, 


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(10, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(10, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(9, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['acc'])

To get recall:

In [ ]:
y_hat_train=model.predict(train_img)
y_hat_test=model.predict(test_img)

Clearly here I have to round to get the actual prediction for the class.

In [ ]:
train_labels

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

pr_train=precision_score(train_labels, train_labels, average=None)
#rec_train=recall_score(train_labels, y_hat_train)
#acc_train=accuracy_score(train_labels, y_hat_train)
#f1_train=f1_score(train_labels, y_hat_train)

#pr_test=precision_score(test_labels, y_hat_test)
#rec_test=recall_score(test_labels, y_hat_test)
#acc_test=accuracy_score(test_labels, y_hat_test)
#f1_test=f1_score(test_labels, y_hat_test)
pr_train
#print("Train: \nprecision", pr_train,"\nrecall:", rec_train, '\naccuracy:', acc_train, '\nf1 score:', f1_train,'\n')
#print("Test: \nprecision", pr_test,"\nrecall:", rec_test, '\naccuracy:', acc_test, '\nf1 score:', f1_test)